In [1]:
!python --version
!which python

Python 3.12.4
/public/workspace/ryrl/venvs/versions/3.12.4/cancer/bin/python


In [2]:
import os
import numpy as np
import pandas as pd


In [3]:
cancers = 'ACC,BLCA,BRCA,CESC,CHOL,COAD,ESCA,GBM,HNSC,KICH,KIRC,KIRP,LGG,LIHC,LUAD,LUSC,OV,PAAD,PRAD,READ,SARC,SKCM,STAD,STES,TGCT,THCA,THYM,UCEC,UCS'.split(',')

In [4]:
df = pd.read_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/GTEx/GTEx_Analysis_v10_Annotations_SampleAttributesDS.txt', sep='\t', header=0, low_memory=False).iloc[:, 0:3]
df['Type'] = df['Type'].str.split('/')
df = df.explode('Type').query(f'Type in {cancers}').reset_index(drop=True)
df = df.assign(**{'Group': 'Normal', 'DataBase': 'GTEx'})
df.rename(columns={'Type': 'Cancer', 'SAMPID': 'Sample'}, inplace=True)
df.head()

,Sample,Cancer,SMTS,Group,DataBase
0,BMS-X4LF-0526-SM-4JBHX,ACC,Adrenal Gland,Normal,GTEx
1,BMS-X4LF-1126-SM-4JBH8,ACC,Adrenal Gland,Normal,GTEx
2,BMS-X4LF-1726-SM-4JBH1,ACC,Adrenal Gland,Normal,GTEx
3,BMS-X4LF-2326-SM-4JBH2,ACC,Adrenal Gland,Normal,GTEx
4,BMS-X4LF-3426-SM-4JBHU,ACC,Adrenal Gland,Normal,GTEx


In [5]:
df.shape

(32190, 5)

In [6]:
df['SMTS'].str.lower().str.replace(' ', '_').unique()

array(['adrenal_gland', 'bladder', 'brain', 'breast', 'cervix_uteri',
       'colon', 'esophagus', 'kidney', 'liver', 'lung', 'ovary',
       'pancreas', 'prostate', 'skin', 'stomach', 'testis', 'thyroid',
       'uterus'], dtype=object)

In [8]:
os.chdir('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/GTEx')
for s in df['SMTS'].str.lower().str.replace(' ', '_').unique():
    print(f'https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_{s}.gct.gz')

https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_adrenal_gland.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_bladder.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_brain.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_breast.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_cervix_uteri.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_colon.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_esophagus.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_kidney.gct.gz
https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/counts-by-tissue/gene_reads_v10_liver.gct.gz
https

In [14]:
metaInfo = pd.read_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/Meta/metaInfo.txt', sep='\t', header=0).query('Cancer != "GTEx"')
metaInfo = metaInfo.assign(DataBase='TCGA')
metaInfo.head()

,Sample,Cancer,Group,DataBase
0,TCGA-FU-A3HZ-01,CESC,Tumor,TCGA
1,TCGA-DR-A0ZM-01,CESC,Tumor,TCGA
2,TCGA-IR-A3LB-01,CESC,Tumor,TCGA
3,TCGA-DG-A2KJ-01,CESC,Tumor,TCGA
4,TCGA-C5-A7CM-01,CESC,Tumor,TCGA


In [15]:
metaInfo = pd.concat([metaInfo, df], axis=0).reset_index(drop=True)
metaInfo.shape

(41057, 4)

In [ ]:
# metaInfo.to_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/Meta/metaInfo.txt', sep='\t', index=False)

In [15]:
gtex = pd.read_csv('/public/workspace/ryrl/FK/TCGA/RNA-seq/GTEx_Analysis_v10_RNASeQCv2.4.2_gene_reads.gct.gz', sep='\t', header=0, low_memory=False, skiprows=2)
gtex.head()

,Name,Description,GTEX-1117F-0005-SM-HL9SH,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-1117F-0011-R11b-SM-GIN8R,GTEX-1117F-0011-R2b-SM-GI4VL,GTEX-1117F-0011-R3a-SM-GJ3PJ,GTEX-1117F-0011-R4b-SM-GI4VM,GTEX-1117F-0011-R5a-SM-GI4VW,GTEX-1117F-0011-R6a-SM-GI4VX,...,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2326-SM-GOQYU,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2526-SM-GOQZ3,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000223972.5,DDX11L1,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,1,1,1
1,ENSG00000227232.5,WASH7P,54,117,457,183,167,223,202,224,...,135,79,86,81,47,89,83,220,32,66
2,ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000243485.5,MIR1302-2HG,0,2,1,0,0,1,0,0,...,1,0,2,2,0,1,1,0,0,0
4,ENSG00000237613.2,FAM138A,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
gtex = gtex.assign(Name=gtex['Name'].str.split('.').str[0])
gtex.drop(columns=['Description'], inplace=True)
gtex = gtex.groupby('Name').median().astype(int)
gtex.head()

,GTEX-1117F-0005-SM-HL9SH,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-1117F-0011-R11b-SM-GIN8R,GTEX-1117F-0011-R2b-SM-GI4VL,GTEX-1117F-0011-R3a-SM-GJ3PJ,GTEX-1117F-0011-R4b-SM-GI4VM,GTEX-1117F-0011-R5a-SM-GI4VW,GTEX-1117F-0011-R6a-SM-GI4VX,GTEX-1117F-0011-R7a-SM-H65ZK,GTEX-1117F-0226-SM-5GZZ7,...,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2326-SM-GOQYU,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2526-SM-GOQZ3,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
Name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,20,1322,514,1483,1250,1348,2421,2827,2096,2255,...,1473,1142,553,7515,2502,4114,574,1537,245,3501
ENSG00000000005,0,4,2,6,31,11,2,3,2,428,...,8,6,5,78,0,61,4,504,302,1147
ENSG00000000419,98,1018,1091,827,790,834,1355,1520,1146,1220,...,1076,1281,1490,1336,1069,1822,1441,1752,1308,1331
ENSG00000000457,59,370,580,244,365,478,532,483,404,658,...,1030,700,756,922,661,1446,756,1634,948,1187
ENSG00000000460,14,94,296,122,79,213,195,233,128,172,...,260,127,240,319,300,368,182,332,98,381


In [44]:
gtex.to_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/GTEx/Gtex_median.txt', sep='\t')

In [46]:
gtex.to_parquet('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/GTEx/Gtex_median.parquet', index=True)

In [21]:
df['Sample'].unique().shape

(29310,)

In [16]:
metaInfo = pd.read_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/Meta/metaInfo.txt', sep='\t', header=0)
metaInfo.head()

,Sample,Cancer,Group,DataBase
0,TCGA-FU-A3HZ-01,CESC,Tumor,TCGA
1,TCGA-DR-A0ZM-01,CESC,Tumor,TCGA
2,TCGA-IR-A3LB-01,CESC,Tumor,TCGA
3,TCGA-DG-A2KJ-01,CESC,Tumor,TCGA
4,TCGA-C5-A7CM-01,CESC,Tumor,TCGA


In [26]:
samples = metaInfo.query('DataBase == "GTEx" and Cancer == "ACC"')['Sample'].values
len(samples)

733

In [24]:
metaInfo.query('Cancer == "ACC"')['DataBase'].unique()

array(['TCGA', 'GTEx'], dtype=object)

In [35]:
demo = gtex[gtex.columns.intersection(samples)]
demo.head()

,GTEX-111CU-0126-SM-5GZWZ,GTEX-111YS-0126-SM-5987T,GTEX-1122O-0326-SM-5H124,GTEX-117YX-0126-SM-5EGH5,GTEX-11DXX-0126-SM-5EGH7,GTEX-11DXY-1626-SM-5H12L,GTEX-11DXZ-0226-SM-5EGGZ,GTEX-11EM3-0326-SM-5A5KJ,GTEX-11EMC-0526-SM-5EGJN,GTEX-11EQ9-0126-SM-5986I,...,GTEX-ZTSS-0326-SM-5987M,GTEX-ZUA1-1926-SM-5E45E,GTEX-ZV7C-1126-SM-5E45O,GTEX-ZVP2-0126-SM-5NQ7D,GTEX-ZVT2-0826-SM-5GIEO,GTEX-ZVZP-1326-SM-5NQ7F,GTEX-ZXES-0226-SM-HLJRM,GTEX-ZY6K-0126-SM-5MR7N,GTEX-ZYFG-0926-SM-5BC5U,GTEX-ZZPU-1226-SM-5N9CK
Name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,1728,1604,2282,1284,1546,1452,1521,2425,840,1672,...,2341,1114,2020,2443,2184,1807,2112,1876,4030,2279
ENSG00000000005,4,0,0,4,0,14,1,1,8,0,...,0,4,2,2,0,2,0,0,7,0
ENSG00000000419,1775,1621,2037,1191,1285,1750,1721,2349,2676,1365,...,1802,1669,2125,1784,1982,1600,2321,1459,3597,2082
ENSG00000000457,914,790,1142,473,665,806,713,916,602,749,...,827,653,952,1036,733,945,784,812,2099,1221
ENSG00000000460,144,156,238,148,148,210,151,122,168,140,...,148,165,183,213,224,192,172,183,404,278


In [37]:
df_ = pd.read_csv('/public/workspace/ryrl/projects/classmates/ryrl/Cancers/TCGA/Counts/ACC.txt', sep='\t', header=0, low_memory=False).set_index('Tag')
df_.head()

,TCGA-OR-A5J5-01,TCGA-OR-A5K4-01,TCGA-OR-A5KU-01,TCGA-OR-A5JK-01,TCGA-OR-A5J8-01,TCGA-OR-A5KW-01,TCGA-OR-A5L9-01,TCGA-OR-A5KY-01,TCGA-OR-A5J3-01,TCGA-PA-A5YG-01,...,TCGA-OR-A5LM-01,TCGA-PK-A5HB-01,TCGA-OR-A5L3-01,TCGA-OR-A5LS-01,TCGA-OR-A5LL-01,TCGA-OR-A5L5-01,TCGA-OR-A5K3-01,TCGA-OR-A5JL-01,TCGA-OR-A5JC-01,TCGA-OR-A5JT-01
Tag,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,1457,2333,1837,1358,1728,2527,220,1037,3543,3109,...,1386,2732,714,1954,505,3823,1419,3474,1798,2452
ENSG00000000005,0,2,1,4,3,3,0,4,1,5,...,0,6,3,23,0,3,3,12,2,20
ENSG00000000419,406,2439,1679,1381,2208,1193,86,2317,1050,2535,...,1759,3179,674,1541,1263,2923,1067,1642,2215,2604
ENSG00000000457,305,368,246,409,536,371,52,429,395,504,...,298,673,271,689,419,613,114,598,236,485
ENSG00000000460,75,177,87,120,417,123,13,343,98,237,...,49,500,35,124,95,176,43,95,107,205


In [43]:
demo_ = pd.concat([df_, demo], axis=1, join='outer').fillna(0).astype(int)
demo_.head()

,TCGA-OR-A5J5-01,TCGA-OR-A5K4-01,TCGA-OR-A5KU-01,TCGA-OR-A5JK-01,TCGA-OR-A5J8-01,TCGA-OR-A5KW-01,TCGA-OR-A5L9-01,TCGA-OR-A5KY-01,TCGA-OR-A5J3-01,TCGA-PA-A5YG-01,...,GTEX-ZTSS-0326-SM-5987M,GTEX-ZUA1-1926-SM-5E45E,GTEX-ZV7C-1126-SM-5E45O,GTEX-ZVP2-0126-SM-5NQ7D,GTEX-ZVT2-0826-SM-5GIEO,GTEX-ZVZP-1326-SM-5NQ7F,GTEX-ZXES-0226-SM-HLJRM,GTEX-ZY6K-0126-SM-5MR7N,GTEX-ZYFG-0926-SM-5BC5U,GTEX-ZZPU-1226-SM-5N9CK
ENSG00000000003,1457,2333,1837,1358,1728,2527,220,1037,3543,3109,...,2341,1114,2020,2443,2184,1807,2112,1876,4030,2279
ENSG00000000005,0,2,1,4,3,3,0,4,1,5,...,0,4,2,2,0,2,0,0,7,0
ENSG00000000419,406,2439,1679,1381,2208,1193,86,2317,1050,2535,...,1802,1669,2125,1784,1982,1600,2321,1459,3597,2082
ENSG00000000457,305,368,246,409,536,371,52,429,395,504,...,827,653,952,1036,733,945,784,812,2099,1221
ENSG00000000460,75,177,87,120,417,123,13,343,98,237,...,148,165,183,213,224,192,172,183,404,278


In [42]:
demo_.shape

(54532, 374)